# wrote by : **Idriss Khattabi**, framed by : Pr . **ELAACHAk LOTFI**

----------------
# **Lab 1 of NLP** : Scraping and NLP Pipeline for Arabic Web Sources

##### The objective of this project is to gain familiarity with scraping and NLP pipeline techniques. Specifically, we aim to scrape data from Arabic web sources related to a specific domain, store the raw data in a NoSQL database (MongoDB), and then establish an NLP pipeline including text cleaning, tokenization, stop words removal, discretization, normalization, stemming, lemmatization, parts of speech tagging using rule-based and machine learning approaches, and named entity recognition (NER) methods.

Work to do :
1. Use libraries like Scrapy and Beautiful Soup to do scraping from server Arabic web sources
concerning a specific domaine.
2. Store the raw data on a NoSql database “MongoDB”.
3. Establishment of NLP Pipeline (Text Cleaning, Tokenization, Stop words, Discretization,
Normalization).
4. Do the Stemming and Lemmatization, then compare the tow mechanisms.
5. Apply Parts of Speech technics based on both Rule based and Machine learning approaches.
5. Apply NER Methods

---------------
## 1. Use Beautiful Soup library for Data Scraping from **hespress.com** : (Data Acquisition)

In [133]:
# Import libraries for Data Scraping
import requests
from bs4 import BeautifulSoup
import csv
from pandas import DataFrame

In [130]:
# Scrap the Data and store it in a dict
articles_info = []

url = f"https://www.hespress.com/societe"

result = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

soup = BeautifulSoup(result.content, "lxml")

articles_container = soup.find("div", {"id": "main-page-content"})

all_articles = articles_container.find_all("div", {"class": "cover"})

print("len(all_articles) : " ,len(all_articles))


len(all_articles) :  12


In [131]:
for article in all_articles :
    link_title = article.find("div", {"class" : "card-img-top"})
    link_title = link_title.find("a")
    article_title = link_title.attrs["title"]
    article_link = link_title.attrs["href"]

    result2 = requests.get(article_link, headers={'User-Agent': 'Mozilla/5.0'})
    soup2 = BeautifulSoup(result2.content, "lxml")
    article_content = soup2.find("div", {"class" : "article-content"}).text.strip()

    articles_info.append({
        "post_title": article_title,
        "post_content": article_content,
        "post_link": article_link
    })

In [132]:
DataFrame(articles_info).head()

,post_title,post_content,post_link
0,آراء تساند تقسيم تركة المغاربة بالوصية وسط تحذ...,ضمن التقرير الذي تسلمه عزيز أخنوش، رئيس الحكوم...,https://www.hespress.com/%d8%a2%d8%b1%d8%a7%d8...
1,"الشرايبي: حضور ""الإجهاض السري"" في المغرب يلامس...",يتجدد النقاش في أوساط المهتمين بشؤون المرأة وح...,https://www.hespress.com/%d8%a7%d9%84%d8%b4%d8...
2,بنموسى يعد بإبقاء المقاربة التشاركية,أعلن شكيب بنموسى، وزير التربية الوطنية والتعلي...,https://www.hespress.com/%d8%a8%d9%86%d9%85%d9...
3,شبكات الهجرة السرية من المغرب تخطط لزيادة عملي...,تتجه أنظار شبكات الهجرة السرية بالمغرب إلى موع...,https://www.hespress.com/%d8%b4%d8%a8%d9%83%d8...
4,"جمعية موظفي المقاومة تدين ""التخوين""",توصلت جريدة هسبريس الإلكترونية برد جمعية الأعم...,https://www.hespress.com/%d8%ac%d9%85%d8%b9%d9...


In [134]:
# I can store the scraped data in a CSV file
columns = articles_info[0].keys()

with open("hespress-posts.csv", "w", encoding="utf-8-sig", newline="") as csv_file:
    dict_writer = csv.DictWriter(csv_file, columns)
    dict_writer.writeheader()
    dict_writer.writerows(articles_info)

    print("*"*50, "\n file created \n", "*"*50)

************************************************** 
 file created 
 **************************************************


##### **Synthesis**: In this question, I have learnt how to use Beautiful Soup to extract data from Arabic web sources efficiently.

---------------
## 2. Store the raw data on a NoSql database “MongoDB” :

In [135]:
# Import libraries
from pymongo import MongoClient

In [142]:
# Connect to MongoDB server
client = MongoClient("mongodb://localhost:27017/")

# Access database
db = client["NLP_DB1"]

# Access collection
collection = db["hespress_posts"]

# Insert multiple rows
collection.insert_many(articles_info)

print("DONE !")

DONE !


##### **Synthesis**: In this question, I have Gained experience in storing and loading scraped data in MongoDB, a NoSQL database.

------------------
## 3. Establishment of NLP Pipeline (Text Cleaning, Tokenization, Stop words, Discretization, Normalization) :


In [143]:
# First, load the data from MongoDB

# Connect to MongoDB server
client = MongoClient('mongodb://localhost:27017/')

# Select the database
db = client['NLP_DB1']

# Select the collection
collection = db['hespress_posts']

projection = {'_id': 0, 'post_content': 1}

# Query the collection to retrieve documents
all_posts = collection.find({}, projection)

# Convert the data to pandas´s DataFrame
all_posts = DataFrame(list(all_posts))

all_posts.head()

,post_content
0,ضمن التقرير الذي تسلمه عزيز أخنوش، رئيس الحكوم...
1,يتجدد النقاش في أوساط المهتمين بشؤون المرأة وح...
2,أعلن شكيب بنموسى، وزير التربية الوطنية والتعلي...
3,تتجه أنظار شبكات الهجرة السرية بالمغرب إلى موع...
4,توصلت جريدة هسبريس الإلكترونية برد جمعية الأعم...


In [157]:
# Import libraries for Establishment of NLP Pipeline 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import arabic_reshaper # for text normalization

##### This is an example of the process of Establishment of NLP Pipeline in the 3rd question :

In [144]:
all_posts["post_content"][0]

'ضمن التقرير الذي تسلمه عزيز أخنوش، رئيس الحكومة، نهاية شهر مارس الماضي، من الهيئة المكلفة بمراجعة مدونة الأسرة، المقرر رفعه إلى الملك محمد السادس، كان لافتا وجود مقترح تطرق إلى تقسيم التركة من خلال “تخويل صاحب المال والأصول سلطة اختيار النظام المُطبق، إما الميراث أو الوصية، مع توسيع نطاق الأخيرة لتشمل الأحفاد ورفع القيود المفروضة عليها”. وعلى الرغم من أن هذا المقترح، الذي عدّه كثيرون “تسريبا”، ورد عموما في وثيقة رائجة ولا يتصل بتوصيات رسمية مرفوعة إلى الملك محمد السادس؛ فإن جهات كثيرة التقطته، خصوصا الأقليات الدينية بالمغرب، ونظرت إليه بـ”تقدير” كـ”حل براغماتي” يساعد “في تقسيم التركات وفق منطق الوصية”، وتمكين المغاربة من “اختيار الذي يناسب شكل عيشهم”. الدين والقانون سعيد ناشيد، حقوقي وباحث في الفكر الديني، اعتبر أن “مقترح تضمين الوصية ضمن الاختيارات أمام توزيع التركة يكشف فعلا عن حل براغماتي يساهم في التخلص من مأزق فرض تقسيم التركة وفق المنظور الإسلامي على أملاك غير المسلمين، سواء كانوا مسيحيين أو بهائيين أو ملاحدة”، مؤكدا أن “الاشتغال بالوصية سيكون مبدأ قانونيا صرفا، حتى لو كانت وارد

In [167]:
# Remove HTML tags 
cleaned_text = re.sub('<[^<]+?>', '', all_posts["post_content"][0])
# Remove non-Arabic characters, digits, and punctuation
cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', cleaned_text)
# Remove extra spaces
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
cleaned_text

'ضمن التقرير الذي تسلمه عزيز أخنوش، رئيس الحكومة، نهاية شهر مارس الماضي، من الهيئة المكلفة بمراجعة مدونة الأسرة، المقرر رفعه إلى الملك محمد السادس، كان لافتا وجود مقترح تطرق إلى تقسيم التركة من خلال تخويل صاحب المال والأصول سلطة اختيار النظام المُطبق، إما الميراث أو الوصية، مع توسيع نطاق الأخيرة لتشمل الأحفاد ورفع القيود المفروضة عليها وعلى الرغم من أن هذا المقترح، الذي عدّه كثيرون تسريبا، ورد عموما في وثيقة رائجة ولا يتصل بتوصيات رسمية مرفوعة إلى الملك محمد السادس؛ فإن جهات كثيرة التقطته، خصوصا الأقليات الدينية بالمغرب، ونظرت إليه بـتقدير كـحل براغماتي يساعد في تقسيم التركات وفق منطق الوصية، وتمكين المغاربة من اختيار الذي يناسب شكل عيشهم الدين والقانون سعيد ناشيد، حقوقي وباحث في الفكر الديني، اعتبر أن مقترح تضمين الوصية ضمن الاختيارات أمام توزيع التركة يكشف فعلا عن حل براغماتي يساهم في التخلص من مأزق فرض تقسيم التركة وفق المنظور الإسلامي على أملاك غير المسلمين، سواء كانوا مسيحيين أو بهائيين أو ملاحدة، مؤكدا أن الاشتغال بالوصية سيكون مبدأ قانونيا صرفا، حتى لو كانت واردة صراحة في النص ا

In [146]:
tokens = word_tokenize(cleaned_text)
print(tokens)
len(tokens)

['ضمن', 'التقرير', 'الذي', 'تسلمه', 'عزيز', 'أخنوش،', 'رئيس', 'الحكومة،', 'نهاية', 'شهر', 'مارس', 'الماضي،', 'من', 'الهيئة', 'المكلفة', 'بمراجعة', 'مدونة', 'الأسرة،', 'المقرر', 'رفعه', 'إلى', 'الملك', 'محمد', 'السادس،', 'كان', 'لافتا', 'وجود', 'مقترح', 'تطرق', 'إلى', 'تقسيم', 'التركة', 'من', 'خلال', 'تخويل', 'صاحب', 'المال', 'والأصول', 'سلطة', 'اختيار', 'النظام', 'المُطبق،', 'إما', 'الميراث', 'أو', 'الوصية،', 'مع', 'توسيع', 'نطاق', 'الأخيرة', 'لتشمل', 'الأحفاد', 'ورفع', 'القيود', 'المفروضة', 'عليها', 'وعلى', 'الرغم', 'من', 'أن', 'هذا', 'المقترح،', 'الذي', 'عدّه', 'كثيرون', 'تسريبا،', 'ورد', 'عموما', 'في', 'وثيقة', 'رائجة', 'ولا', 'يتصل', 'بتوصيات', 'رسمية', 'مرفوعة', 'إلى', 'الملك', 'محمد', 'السادس؛', 'فإن', 'جهات', 'كثيرة', 'التقطته،', 'خصوصا', 'الأقليات', 'الدينية', 'بالمغرب،', 'ونظرت', 'إليه', 'بـتقدير', 'كـحل', 'براغماتي', 'يساعد', 'في', 'تقسيم', 'التركات', 'وفق', 'منطق', 'الوصية،', 'وتمكين', 'المغاربة', 'من', 'اختيار', 'الذي', 'يناسب', 'شكل', 'عيشهم', 'الدين', 'والقانون', 'سعيد', 

662

In [189]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\driss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [184]:
print("Arabic stop-words : ", set(stopwords.words('arabic')))

Arabic stop-words :  {'حدَث', 'سبحان', 'استحال', 'لما', 'دون', 'فلان', 'بسّ', 'ولكن', 'تين', 'ثمة', 'ضحوة', 'مايو', 'كليهما', 'هاء', 'اللتان', 'إياكم', 'أينما', 'إذ', 'ذان', 'كن', 'الألى', 'خال', 'أم', 'مليم', 'لهن', 'هل', 'ما', 'ذِه', 'ثلاثين', 'حمٌ', 'ذلك', 'ح', 'لهما', 'خاء', 'ءَ', 'صباح', 'أيّان', 'بي', 'ذَيْنِ', 'حمو', 'بس', 'وما', 'سبتمبر', 'هيا', 'سرعان', 'تسعين', 'إياكما', 'هَجْ', 'جعل', 'لي', 'سبت', 'كذا', 'لن', 'أخبر', 'ليسوا', 'ستون', 'انبرى', 'هاكَ', 'هَذِي', 'حادي', 'زود', 'ل', 'إذا', 'هَيْهات', 'حقا', 'ذاك', 'يوليو', 'تَيْنِ', 'غين', 'أيضا', 'خلا', 'إيه', 'بطآن', 'حرى', 'هؤلاء', 'كيفما', 'ذو', 'اللتيا', 'بما', 'كأن', 'إياها', 'ما برح', 'إنا', 'ثمانمئة', 'أنبأ', 'ط', 'حيث', 'لنا', 'ماذا', 'هو', 'ذانِ', 'وَيْ', 'تلكم', 'غ', 'د', 'خميس', 'لاسيما', 'أكتوبر', 'باء', 'شَتَّانَ', 'أنتم', 'تجاه', 'لعل', 'أوشك', 'ق', 'إلَيْكَ', 'غدا', 'اثنان', 'ه', 'نبَّا', 'ز', 'تفعلين', 'ولا', 'على', 'إلا', 'غداة', 'أربعمائة', 'بيد', 'ثمانون', 'أين', 'خبَّر', 'لبيك', 'ثلاثمئة', 'بنا', 'رزق', 'سن

In [147]:
# Remove stop words
filtered_tokens = [token for token in tokens if token not in set(stopwords.words('arabic'))]
print(filtered_tokens)
len(filtered_tokens)

['ضمن', 'التقرير', 'تسلمه', 'عزيز', 'أخنوش،', 'رئيس', 'الحكومة،', 'نهاية', 'شهر', 'الماضي،', 'الهيئة', 'المكلفة', 'بمراجعة', 'مدونة', 'الأسرة،', 'المقرر', 'رفعه', 'الملك', 'محمد', 'السادس،', 'لافتا', 'وجود', 'مقترح', 'تطرق', 'تقسيم', 'التركة', 'خلال', 'تخويل', 'صاحب', 'المال', 'والأصول', 'سلطة', 'اختيار', 'النظام', 'المُطبق،', 'الميراث', 'الوصية،', 'توسيع', 'نطاق', 'الأخيرة', 'لتشمل', 'الأحفاد', 'ورفع', 'القيود', 'المفروضة', 'عليها', 'وعلى', 'الرغم', 'المقترح،', 'عدّه', 'كثيرون', 'تسريبا،', 'عموما', 'وثيقة', 'رائجة', 'يتصل', 'بتوصيات', 'رسمية', 'مرفوعة', 'الملك', 'محمد', 'السادس؛', 'جهات', 'كثيرة', 'التقطته،', 'خصوصا', 'الأقليات', 'الدينية', 'بالمغرب،', 'ونظرت', 'إليه', 'بـتقدير', 'كـحل', 'براغماتي', 'يساعد', 'تقسيم', 'التركات', 'وفق', 'منطق', 'الوصية،', 'وتمكين', 'المغاربة', 'اختيار', 'يناسب', 'شكل', 'عيشهم', 'الدين', 'والقانون', 'سعيد', 'ناشيد،', 'حقوقي', 'وباحث', 'الفكر', 'الديني،', 'اعتبر', 'مقترح', 'تضمين', 'الوصية', 'ضمن', 'الاختيارات', 'توزيع', 'التركة', 'يكشف', 'فعلا', 'حل', 'ب

503

In [152]:
# Normalize the text
normalized_text = arabic_reshaper.reshape(' '.join(filtered_tokens))
normalized_text

'ﺿﻤﻦ ﺍﻟﺘﻘﺮﻳﺮ ﺗﺴﻠﻤﻪ ﻋﺰﻳﺰ ﺃﺧﻨﻮﺵ، ﺭﺋﻴﺲ ﺍﻟﺤﻜﻮﻣﺔ، ﻧﻬﺎﻳﺔ ﺷﻬﺮ ﺍﻟﻤﺎﺿﻲ، ﺍﻟﻬﻴﺌﺔ ﺍﻟﻤﻜﻠﻔﺔ ﺑﻤﺮﺍﺟﻌﺔ ﻣﺪﻭﻧﺔ ﺍﻷﺳﺮﺓ، ﺍﻟﻤﻘﺮﺭ ﺭﻓﻌﻪ ﺍﻟﻤﻠﻚ ﻣﺤﻤﺪ ﺍﻟﺴﺎﺩﺱ، ﻻﻓﺘﺎ ﻭﺟﻮﺩ ﻣﻘﺘﺮﺡ ﺗﻄﺮﻕ ﺗﻘﺴﻴﻢ ﺍﻟﺘﺮﻛﺔ ﺧﻼﻝ ﺗﺨﻮﻳﻞ ﺻﺎﺣﺐ ﺍﻟﻤﺎﻝ ﻭﺍﻷﺻﻮﻝ ﺳﻠﻄﺔ ﺍﺧﺘﻴﺎﺭ ﺍﻟﻨﻈﺎﻡ ﺍﻟﻤﻄﺒﻖ، ﺍﻟﻤﻴﺮﺍﺙ ﺍﻟﻮﺻﻴﺔ، ﺗﻮﺳﻴﻊ ﻧﻄﺎﻕ ﺍﻷﺧﻴﺮﺓ ﻟﺘﺸﻤﻞ ﺍﻷﺣﻔﺎﺩ ﻭﺭﻓﻊ ﺍﻟﻘﻴﻮﺩ ﺍﻟﻤﻔﺮﻭﺿﺔ ﻋﻠﻴﻬﺎ ﻭﻋﻠﻰ ﺍﻟﺮﻏﻢ ﺍﻟﻤﻘﺘﺮﺡ، ﻋﺪﻩ ﻛﺜﻴﺮﻭﻥ ﺗﺴﺮﻳﺒﺎ، ﻋﻤﻮﻣﺎ ﻭﺛﻴﻘﺔ ﺭﺍﺋﺠﺔ ﻳﺘﺼﻞ ﺑﺘﻮﺻﻴﺎﺕ ﺭﺳﻤﻴﺔ ﻣﺮﻓﻮﻋﺔ ﺍﻟﻤﻠﻚ ﻣﺤﻤﺪ ﺍﻟﺴﺎﺩﺱ؛ ﺟﻬﺎﺕ ﻛﺜﻴﺮﺓ ﺍﻟﺘﻘﻄﺘﻪ، ﺧﺼﻮﺻﺎ ﺍﻷﻗﻠﻴﺎﺕ ﺍﻟﺪﻳﻨﻴﺔ ﺑﺎﻟﻤﻐﺮﺏ، ﻭﻧﻈﺮﺕ ﺇﻟﻴﻪ ﺑـﺘﻘﺪﻳﺮ ﻛـﺤﻞ ﺑﺮﺍﻏﻤﺎﺗﻲ ﻳﺴﺎﻋﺪ ﺗﻘﺴﻴﻢ ﺍﻟﺘﺮﻛﺎﺕ ﻭﻓﻖ ﻣﻨﻄﻖ ﺍﻟﻮﺻﻴﺔ، ﻭﺗﻤﻜﻴﻦ ﺍﻟﻤﻐﺎﺭﺑﺔ ﺍﺧﺘﻴﺎﺭ ﻳﻨﺎﺳﺐ ﺷﻜﻞ ﻋﻴﺸﻬﻢ ﺍﻟﺪﻳﻦ ﻭﺍﻟﻘﺎﻧﻮﻥ ﺳﻌﻴﺪ ﻧﺎﺷﻴﺪ، ﺣﻘﻮﻗﻲ ﻭﺑﺎﺣﺚ ﺍﻟﻔﻜﺮ ﺍﻟﺪﻳﻨﻲ، ﺍﻋﺘﺒﺮ ﻣﻘﺘﺮﺡ ﺗﻀﻤﻴﻦ ﺍﻟﻮﺻﻴﺔ ﺿﻤﻦ ﺍﻻﺧﺘﻴﺎﺭﺍﺕ ﺗﻮﺯﻳﻊ ﺍﻟﺘﺮﻛﺔ ﻳﻜﺸﻒ ﻓﻌﻼ ﺣﻞ ﺑﺮﺍﻏﻤﺎﺗﻲ ﻳﺴﺎﻫﻢ ﺍﻟﺘﺨﻠﺺ ﻣﺄﺯﻕ ﻓﺮﺽ ﺗﻘﺴﻴﻢ ﺍﻟﺘﺮﻛﺔ ﻭﻓﻖ ﺍﻟﻤﻨﻈﻮﺭ ﺍﻹﺳﻼﻣﻲ ﺃﻣﻼﻙ ﺍﻟﻤﺴﻠﻤﻴﻦ، ﺳﻮﺍﺀ ﻛﺎﻧﻮﺍ ﻣﺴﻴﺤﻴﻴﻦ ﺑﻬﺎﺋﻴﻴﻦ ﻣﻼﺣﺪﺓ، ﻣﺆﻛﺪﺍ ﺍﻻﺷﺘﻐﺎﻝ ﺑﺎﻟﻮﺻﻴﺔ ﺳﻴﻜﻮﻥ ﻣﺒﺪﺃ ﻗﺎﻧﻮﻧﻴﺎ ﺻﺮﻓﺎ، ﻛﺎﻧﺖ ﻭﺍﺭﺩﺓ ﺍﻟﻨﺺ ﺍﻟﺪﻳﻨﻲ ﺍﻟﻘﺮﺁﻧﻲ ﻭﻭﺿﺢ ﻧﺎﺷﻴﺪ، ﺗﺼﺮﻳﺢ ﻟﻬﺴﺒﺮﻳﺲ، ﺍﻟﻮﺻﻴﺔ ﺗﻤﺜﻞ ﻣﺨﺮﺟﺎ ﻭﺣﻼ ﻟﻤﺸﻜﻞ ﺍﻹﺭﺙ ﺩﺍﺧﻞ ﺑﻨﻴﺔ ﺍﻟﺪﻳﻦ ﻧﻔﺴﻪ، ﻭﻫﻲ ﺩﻟﻴﻞ ﺍﻟﺪﻳﻦ ﻭﺍﺳﻊ ﻭﺭﺣﺐ ﻳﻜﻔﻲ ﻟﻨﺠﺪ ﻣﺨﺎﺭﺝ ﻟﻜﻞ ﺍﻟﻤﺸﺎﻛﻞ ﺍﻟﻤ

##### Now, we are going to apply the process on the all data

In [155]:
# Load Arabic stop words
stop_words = set(stopwords.words('arabic'))

# text cleaning function
def clean_text(text):
    # Remove HTML tags 
    cleaned_text = re.sub('<[^<]+?>', '', text)
    # Remove non-Arabic characters, digits, and punctuation
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', cleaned_text)
    # Remove extra spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text

# text normalization function
def normalize_text(text):
    # Normalize Arabic characters
    normalized_text = arabic_reshaper.reshape(text)
    return normalized_text


In [178]:
post_content2 = []

# Process each text
for i in range(len(all_posts)):
    # Get Arabic text from MongoDB document
    arabic_text = all_posts["post_content"][i]
    # print("before : ", arabic_text)
    
    # Clean the text
    cleaned_text = clean_text(arabic_text)
    
    # Tokenize the cleaned text
    tokens = word_tokenize(cleaned_text)
    
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # Normalize the text
    normalized_text = normalize_text(' '.join(filtered_tokens))
    
    # Print or store the processed text
    post_content2.append(normalized_text)
    # print("after : ", normalized_text)
    # print("*"*50)

all_posts["post_content2"] = post_content2

In [159]:
all_posts.head()

,post_content,post_content2
0,ضمن التقرير الذي تسلمه عزيز أخنوش، رئيس الحكوم...,ﺿﻤﻦ ﺍﻟﺘﻘﺮﻳﺮ ﺗﺴﻠﻤﻪ ﻋﺰﻳﺰ ﺃﺧﻨﻮﺵ، ﺭﺋﻴﺲ ﺍﻟﺤﻜﻮﻣﺔ، ﻧﻬ...
1,يتجدد النقاش في أوساط المهتمين بشؤون المرأة وح...,ﻳﺘﺠﺪﺩ ﺍﻟﻨﻘﺎﺵ ﺃﻭﺳﺎﻁ ﺍﻟﻤﻬﺘﻤﻴﻦ ﺑﺸﺆﻭﻥ ﺍﻟﻤﺮﺃﺓ ﻭﺣﻘﻮﻗ...
2,أعلن شكيب بنموسى، وزير التربية الوطنية والتعلي...,ﺃﻋﻠﻦ ﺷﻜﻴﺐ ﺑﻨﻤﻮﺳﻰ، ﻭﺯﻳﺮ ﺍﻟﺘﺮﺑﻴﺔ ﺍﻟﻮﻃﻨﻴﺔ ﻭﺍﻟﺘﻌﻠﻴ...
3,تتجه أنظار شبكات الهجرة السرية بالمغرب إلى موع...,ﺗﺘﺠﻪ ﺃﻧﻈﺎﺭ ﺷﺒﻜﺎﺕ ﺍﻟﻬﺠﺮﺓ ﺍﻟﺴﺮﻳﺔ ﺑﺎﻟﻤﻐﺮﺏ ﻣﻮﻋﺪ ﻋﻴ...
4,توصلت جريدة هسبريس الإلكترونية برد جمعية الأعم...,ﺗﻮﺻﻠﺖ ﺟﺮﻳﺪﺓ ﻫﺴﺒﺮﻳﺲ ﺍﻹﻟﻜﺘﺮﻭﻧﻴﺔ ﺑﺮﺩ ﺟﻤﻌﻴﺔ ﺍﻷﻋﻤﺎﻝ...


##### **Synthesis**: In this question, I have Established an NLP pipeline involving text cleaning, tokenization, stop words removal and normalization for Arabic text.

-------------------
## 4. Do the Stemming and Lemmatization, then compare the tow mechanisms : 

**Comparison** : The main difference between lemmatization and stemming is the level of abstraction they provide when reducing words to their base forms:

1. Lemmatization:
   - Lemmatization aims to return the base or dictionary form of a word, known as the lemma, by considering the context and part of speech of the word.
   - It typically involves dictionary lookup and morphological analysis to accurately determine the lemma.
   - Lemmatization results in more accurate base forms compared to stemming.
   - It requires more computational resources and time compared to stemming.
   - Lemmatization produces real words or lemmas that have meaning in the language.

2. Stemming:
   - Stemming is a simpler and faster process that chops off affixes from words to get to the root or stem.
   - It does not consider the context or part of speech of the word, leading to potential inaccuracies.
   - Stemming may produce non-real words or stems that may not have meaning in the language.
   - Stemming is less computationally intensive compared to lemmatization.

In [160]:
from nltk.stem import WordNetLemmatizer, ISRIStemmer
from nltk.tokenize import word_tokenize
import nltk

In [180]:
# Arabic stemmer
stemmer = ISRIStemmer()

# WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

post_content3 = []

for i in range(len(all_posts)):

    arabic_text = all_posts["post_content2"][i]

    # Tokenize the sentence
    tokens = word_tokenize(arabic_text)

    # Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(token) for token in tokens]

    # Stemming
    stemmed_words = [stemmer.stem(token) for token in tokens]

    # store the lemmatized words because Lemmatization return the base or dictionary form of a word,
    # known as the lemma, by considering the context and part of speech of the word.
    post_content3.append(" ".join(lemmatized_words))

    if i == 1 :
        print("arabic_text : ", arabic_text)
        print("-"*50)
        print("Lemmatized words:", lemmatized_words)
        print("-"*50)
        print("Stemmed words:", stemmed_words)


all_posts["post_content3"] = post_content3


arabic_text :  ﻳﺘﺠﺪﺩ ﺍﻟﻨﻘﺎﺵ ﺃﻭﺳﺎﻁ ﺍﻟﻤﻬﺘﻤﻴﻦ ﺑﺸﺆﻭﻥ ﺍﻟﻤﺮﺃﺓ ﻭﺣﻘﻮﻗﻬﺎ ﺍﻟﻤﻐﺮﺏ، ﻇﻞ ﺍﻟﺤﺪﻳﺚ ﺍﻟﺤﺎﻟﻲ ﺗﻌﺪﻳﻞ ﻣﺪﻭﻧﺔ ﺍﻷﺳﺮﺓ، ﺃﻳﻀﺎ، ﺣﻮﻝ ﺗﻘﻨﻴﻦ ﺍﻹﺟﻬﺎﺽ ﻭﻓﻲ ﺍﻹﻃﺎﺭ، ﻃﺎﻟﺐ ﺷﻔﻴﻖ ﺍﻟﺸﺮﺍﻳﺒﻲ، ﺭﺋﻴﺲ ﺍﻟﺠﻤﻌﻴﺔ ﺍﻟﻤﻐﺮﺑﻴﺔ ﻟﻤﺤﺎﺭﺑﺔ ﺍﻹﺟﻬﺎﺽ ﺍﻟﺴﺮﻱ، ﺍﻟﺤﻜﻮﻣﺔ ﺍﻟﺤﺎﻟﻴﺔ ﺑﻀﺮﻭﺭﺓ ﺗﻘﻨﻴﻨﻪ، ﻣﻌﺘﺒﺮﺍ ﺃﻧﻪ ﻳﺰﺍﻝ ﺇﺷﻜﺎﻻ ﻳﻌﺘﺮﺽ ﺍﻟﻤﺠﺘﻤﻊ ﺍﻟﻤﻐﺮﺑﻲ ﻇﻞ ﺗﺴﺠﻴﻞ ﺣﻮﺍﻟﻲ ﺣﺎﻟﺔ ﺇﺟﻬﺎﺽ ﻳﻮﻣﻴﺎ ﻭﻗﺎﻝ ﺭﺋﻴﺲ ﺍﻟﺠﻤﻌﻴﺔ ﺍﻟﻤﻐﺮﺑﻴﺔ ﻟﻤﺤﺎﺭﺑﺔ ﺍﻹﺟﻬﺎﺽ ﺍﻟﺴﺮﻱ، ﺿﻤﻦ ﺗﺼﺮﻳﺢ ﻟﻬﺴﺒﺮﻳﺲ، ﺗﻄﺎﻟﺐ ﺟﻤﻌﻴﺘﻪ ﺍﻟﺘﻮﻗﻒ ﺍﻵﻣﻦ ﻟﻠﺤﻤﻞ، ﻭﺃﻧﻬﺎ ﺗﺸﺠﻊ ﺍﻹﺟﻬﺎﺽ، ﻣﺸﻴﺮﺍ ﺃﻧﻪ ﻟﻐﺔ ﺍﻷﺭﻗﺎﻡ ﺍﻹﺟﻬﺎﺽ ﺃﻣﺮ ﺳﺮﻱ؛ ﺳﺒﻖ ﻗﻤﻨﺎ ﺑﺒﺤﺚ ﺳﺮﻱ ﻭﺗﻮﺻﻠﻨﺎ ﺃﻧﻪ ﺍﻟﺮﺑﺎﻁ ﻭﺳﻼ ﺣﺎﻟﺔ ﺇﺟﻬﺎﺽ ﻳﻮﻣﻴﺎ، ﻭﺍﻟﺪﺍﺭ ﺍﻟﺒﻴﻀﺎﺀ ﻷﻧﻬﺎ ﺃﻛﺒﺮ ﺣﻮﺍﻟﻲ ﺣﺎﻟﺔ ﺇﺟﻬﺎﺽ، ﺑﺎﻗﻲ ﺍﻟﻤﺪﻥ ﺍﻷﺧﺮﻯ، ﻟﻴﺼﻞ ﺍﻟﺮﻗﻢ ﺣﻮﺍﻟﻲ ﺣﺎﻟﺔ ﻳﻮﻣﻴﺎ ﻭﺃﺿﺎﻑ ﺍﻟﺸﺮﺍﻳﺒﻲ ﺍﻟﻨﺎﺣﻴﺔ ﺩﻳﻨﻴﺔ، ﻭﺟﻮﺩ ﻷﻱ ﺁﻳﺔ ﻗﺮﺁﻧﻴﺔ ﺣﺪﻳﺚ ﺷﺮﻳﻒ ﺗﺘﺤﺪﺙ ﺍﻹﺟﻬﺎﺽ، ﻣﺘﺎﺑﻌﺎ ﺍﻹﺟﻬﺎﺽ ﻣﻘﺒﻮﻝ ﻃﺮﻑ ﺍﻟﻘﺎﻧﻮﻥ ﺍﻟﻨﺴﺎﺀ ﻳﻠﺠﺄﻥ ﺍﻹﺟﻬﺎﺽ ﺍﻟﺴﺮﻱ؛ ﺃﻣﺮ ﺧﻄﻴﺮ ﺟﺪﺍ ﻭﻣﻤﺎ ﻳﺠﻌﻞ ﺧﻄﻴﺮﺍ، ﻗﺎﻝ ﺭﺋﻴﺲ ﺍﻟﺠﻤﻌﻴﺔ ﺍﻟﻤﻐﺮﺑﻴﺔ ﻟﻤﺤﺎﺭﺑﺔ ﺍﻹﺟﻬﺎﺽ ﺍﻟﺴﺮﻱ، ﻷﻧﻪ ﺗﻠﺠﺄ ﺍﻟﻨﺴﺎﺀ ﺍﻷﻃﺒﺎﺀ ﻳﻘﻮﻣﻮﻥ ﺑﺴﺮﻳﺔ ﺑﺜﻤﻦ ﻣﺮﺗﻔﻊ، ﺇﺟﻬﺎﺽ ﺁﻣﻦ ﺍﻟﻤﺎﺋﺔ ﺍﻹﺟﻬﺎﺽ ﺍﻟﺘﻘﻠﻴﺪﻱ، ﺧﻄﻮﺭﺓ ﻷﻥ ﺗﻘﻮﻡ ﺗﺤﺎﻭﻝ ﺗﺠﻬﺾ ﻧﻔﺴﻬﺎ ﺑﻨﻔﺴﻬﺎ، ﺗﺘﺮﺗﺐ ﻋﻨﻪ ﻣﻀﺎﻋﻔﺎﺕ ﻛﺜﻴﺮﺓ؛ ﻣﺜﻼ ﻧﺰﻳﻒ، ﺗﻤﺰﻗﺎﺕ ﺍﻟﺮﺣﻢ، ﺗﻌﻔ

In [100]:
all_posts.head()

,post_content,post_content2,post_content3
0,علمت جريدة هسبريس من قيادات نقابيّة أن “الحكوم...,ﻋﻠﻤﺖ ﺟﺮﻳﺪﺓ ﻫﺴﺒﺮﻳﺲ ﻗﻴﺎﺩﺍﺕ ﻧﻘﺎﺑﻴﺔ ﺍﻟﺤﻜﻮﻣﺔ ﺷﺮﻋﺖ ﺇ...,ﻋﻠﻤﺖ ﺟﺮﻳﺪﺓ ﻫﺴﺒﺮﻳﺲ ﻗﻴﺎﺩﺍﺕ ﻧﻘﺎﺑﻴﺔ ﺍﻟﺤﻜﻮﻣﺔ ﺷﺮﻋﺖ ﺇ...
1,دافع شكيب بنموسى، وزير التربية الوطنية والتعلي...,ﺩﺍﻓﻊ ﺷﻜﻴﺐ ﺑﻨﻤﻮﺳﻰ، ﻭﺯﻳﺮ ﺍﻟﺘﺮﺑﻴﺔ ﺍﻟﻮﻃﻨﻴﺔ ﻭﺍﻟﺘﻌﻠﻴ...,ﺩﺍﻓﻊ ﺷﻜﻴﺐ ﺑﻨﻤﻮﺳﻰ، ﻭﺯﻳﺮ ﺍﻟﺘﺮﺑﻴﺔ ﺍﻟﻮﻃﻨﻴﺔ ﻭﺍﻟﺘﻌﻠﻴ...
2,أطلقت فدرالية رابطة حقوق النساء حملة تواصلية ت...,ﺃﻃﻠﻘﺖ ﻓﺪﺭﺍﻟﻴﺔ ﺭﺍﺑﻄﺔ ﺣﻘﻮﻕ ﺍﻟﻨﺴﺎﺀ ﺣﻤﻠﺔ ﺗﻮﺍﺻﻠﻴﺔ ﺗ...,ﺃﻃﻠﻘﺖ ﻓﺪﺭﺍﻟﻴﺔ ﺭﺍﺑﻄﺔ ﺣﻘﻮﻕ ﺍﻟﻨﺴﺎﺀ ﺣﻤﻠﺔ ﺗﻮﺍﺻﻠﻴﺔ ﺗ...
3,يبدو أن تكلفة المعيشة في المغرب مقبولة مقارنة ...,ﻳﺒﺪﻭ ﺗﻜﻠﻔﺔ ﺍﻟﻤﻌﻴﺸﺔ ﺍﻟﻤﻐﺮﺏ ﻣﻘﺒﻮﻟﺔ ﻣﻘﺎﺭﻧﺔ ﺩﻭﻝ ﺍﻟ...,ﻳﺒﺪﻭ ﺗﻜﻠﻔﺔ ﺍﻟﻤﻌﻴﺸﺔ ﺍﻟﻤﻐﺮﺏ ﻣﻘﺒﻮﻟﺔ ﻣﻘﺎﺭﻧﺔ ﺩﻭﻝ ﺍﻟ...
4,في ظل الاكتظاظ الكبير الذي تعاني منه المؤسسات ...,ﻇﻞ ﺍﻻﻛﺘﻈﺎﻅ ﺍﻟﻜﺒﻴﺮ ﺗﻌﺎﻧﻲ ﺍﻟﻤﺆﺳﺴﺎﺕ ﺍﻟﺴﺠﻨﻴﺔ ﺑﺎﻟﻤﻤ...,ﻇﻞ ﺍﻻﻛﺘﻈﺎﻅ ﺍﻟﻜﺒﻴﺮ ﺗﻌﺎﻧﻲ ﺍﻟﻤﺆﺳﺴﺎﺕ ﺍﻟﺴﺠﻨﻴﺔ ﺑﺎﻟﻤﻤ...


    In summary, the choice between stemming and lemmatization depends on the specific requirements of the NLP task. While stemming is suitable for general text processing tasks that prioritize speed and simplicity, lemmatization is preferred for applications where accuracy and semantic fidelity are paramount.

##### **Synthesis**: In this question, I have Implemented stemming and lemmatization techniques and compared their effectiveness for Arabic text processing.

--------------
## 5. Apply Parts of Speech technics based on both Rule based and Machine learning approaches.

##### **Rule-based Approach**: In the rule-based approach, we define patterns or rules to tag each word based on its context.
##### **ML Approach**: For the machine learning approach, we can use pre-trained models or train our own model on annotated data.

##### For Rule-based Approach :

In [185]:
# Define rules for part-of-speech tagging using regular expressions
patterns = [
    (r'^ال.*$', 'اسم'),            # Nouns starting with ال
    (r'^و.*$', 'اسم'),              # Nouns starting with و
    (r'^ب.*$', 'اسم'),              # Nouns starting with ب
    (r'^ف.*$', 'اسم'),              # Nouns starting with ف
    (r'^ك.*$', 'اسم'),              # Nouns starting with ك
    (r'^ل.*$', 'اسم'),              # Nouns starting with ل
    (r'^م.*$', 'اسم'),              # Nouns starting with م
    (r'^ن.*$', 'اسم'),              # Nouns starting with ن
    (r'^ت.*$', 'اسم'),              # Nouns starting with ت
    (r'^ه.*$', 'اسم'),              # Nouns starting with ه
    (r'^ي.*$', 'اسم'),              # Nouns starting with ي
    (r'^ع.*$', 'اسم'),              # Nouns starting with ع
    (r'^ص.*$', 'اسم'),              # Nouns starting with ص
    (r'^ح.*$', 'اسم'),              # Nouns starting with ح
    (r'^ق.*$', 'اسم'),              # Nouns starting with ق
    (r'^ر.*$', 'اسم'),              # Nouns starting with ر
    (r'^س.*$', 'اسم'),              # Nouns starting with س
    (r'^ش.*$', 'اسم'),              # Nouns starting with ش
    (r'^ض.*$', 'اسم'),              # Nouns starting with ض
    (r'^ظ.*$', 'اسم'),              # Nouns starting with ظ
    (r'^غ.*$', 'اسم'),              # Nouns starting with غ
    (r'^هذا$', 'اسم'),             # Specific words like هذا
    (r'^هذه$', 'اسم'),             # Specific words like هذه
    (r'^هو$', 'ضمير'),              # Specific pronouns like هو
    (r'^هي$', 'ضمير'),              # Specific pronouns like هي
    (r'.*ُ$', 'اسم'),               # Nouns ending with ُ
    (r'.*َة$', 'اسم'),              # Nouns ending with َة
    (r'.*ِ$', 'فعل'),               # Verbs ending with ِ
    (r'.*َ$', 'صفة'),               # Adjectives ending with َ
    (r'.*َةُ$', 'اسم'),            # Nouns ending with َةُ
    (r'.*َةِ$', 'اسم'),            # Nouns ending with َةِ
    (r'.*ُون$', 'اسم'),            # Nouns ending with ُون
    (r'.*ِين$', 'اسم'),            # Nouns ending with ِين
    (r'.*ْ$', 'اسم'),              # Nouns ending with ْ
    (r'.*ى$', 'ضمير'),              # Pronouns ending with ى
    (r'.*َه$', 'ضمير'),             # Pronouns ending with ه
    (r'.*ت$', 'فعل'),              # Verbs ending with ت
    (r'.*نا$', 'ضمير'),             # Pronouns ending with نا
    (r'.*ك$', 'ضمير'),              # Pronouns ending with ك
    (r'.*ي$', 'ضمير'),              # Pronouns ending with ي
    (r'.*ُو$', 'ضمير'),             # Pronouns ending with ُو
    (r'.*ها$', 'ضمير'),             # Pronouns ending with ها
    (r'.*كما$', 'ضمير'),            # Pronouns ending with كما
    (r'.*كن$', 'ضمير'),             # Pronouns ending with كن
    (r'.*كم$', 'ضمير'),             # Pronouns ending with كم
    (r'.*ني$', 'ضمير'),             # Pronouns ending with ني
    (r'.*هما$', 'ضمير'),            # Pronouns ending with هما
    (r'.*هم$', 'ضمير'),             # Pronouns ending with هم
    (r'.*هن$', 'ضمير'),             # Pronouns ending with هن
    (r'.*ه$', 'ضمير'),              # Pronouns ending with ه
    (r'.*تا$', 'فعل'),              # Verbs ending with تا
    (r'.*تما$', 'فعل'),             # Verbs ending with تما
    (r'.*وا$', 'فعل'),              # Verbs ending with وا
    (r'.*ن$', 'فعل'),               # Verbs ending with ن
    (r'.*ه$', 'فعل'),               # Verbs ending with ه
    (r'.*و$', 'فعل'),               # Verbs ending with و
    (r'.*ي$', 'فعل'),               # Verbs ending with ي
    (r'^\w+$', 'اسم')               # Default to NOUN if no pattern matches
]

# Compile the patterns into a tagger
regexp_tagger = nltk.RegexpTagger(patterns)

post_content4 = []
for i in range(len(all_posts)):

    arabic_text = all_posts["post_content3"][i]

    # Tokenize the sentence
    tokens = word_tokenize(arabic_text)

    # Tag the words using the rule-based tagger
    tagged_words = regexp_tagger.tag(tokens)

    post_content4.append(tagged_words)


    if i == 1:
        # print("arabic_text : ", tokens)
        print("NLTK-based POS tagging: ", tagged_words)

NLTK-based POS tagging:  [('ﻳﺘﺠﺪﺩ', 'اسم'), ('ﺍﻟﻨﻘﺎﺵ', 'اسم'), ('ﺃﻭﺳﺎﻁ', 'اسم'), ('ﺍﻟﻤﻬﺘﻤﻴﻦ', 'اسم'), ('ﺑﺸﺆﻭﻥ', 'اسم'), ('ﺍﻟﻤﺮﺃﺓ', 'اسم'), ('ﻭﺣﻘﻮﻗﻬﺎ', 'اسم'), ('ﺍﻟﻤﻐﺮﺏ،', None), ('ﻇﻞ', 'اسم'), ('ﺍﻟﺤﺪﻳﺚ', 'اسم'), ('ﺍﻟﺤﺎﻟﻲ', 'اسم'), ('ﺗﻌﺪﻳﻞ', 'اسم'), ('ﻣﺪﻭﻧﺔ', 'اسم'), ('ﺍﻷﺳﺮﺓ،', None), ('ﺃﻳﻀﺎ،', None), ('ﺣﻮﻝ', 'اسم'), ('ﺗﻘﻨﻴﻦ', 'اسم'), ('ﺍﻹﺟﻬﺎﺽ', 'اسم'), ('ﻭﻓﻲ', 'اسم'), ('ﺍﻹﻃﺎﺭ،', None), ('ﻃﺎﻟﺐ', 'اسم'), ('ﺷﻔﻴﻖ', 'اسم'), ('ﺍﻟﺸﺮﺍﻳﺒﻲ،', None), ('ﺭﺋﻴﺲ', 'اسم'), ('ﺍﻟﺠﻤﻌﻴﺔ', 'اسم'), ('ﺍﻟﻤﻐﺮﺑﻴﺔ', 'اسم'), ('ﻟﻤﺤﺎﺭﺑﺔ', 'اسم'), ('ﺍﻹﺟﻬﺎﺽ', 'اسم'), ('ﺍﻟﺴﺮﻱ،', None), ('ﺍﻟﺤﻜﻮﻣﺔ', 'اسم'), ('ﺍﻟﺤﺎﻟﻴﺔ', 'اسم'), ('ﺑﻀﺮﻭﺭﺓ', 'اسم'), ('ﺗﻘﻨﻴﻨﻪ،', None), ('ﻣﻌﺘﺒﺮﺍ', 'اسم'), ('ﺃﻧﻪ', 'اسم'), ('ﻳﺰﺍﻝ', 'اسم'), ('ﺇﺷﻜﺎﻻ', 'اسم'), ('ﻳﻌﺘﺮﺽ', 'اسم'), ('ﺍﻟﻤﺠﺘﻤﻊ', 'اسم'), ('ﺍﻟﻤﻐﺮﺑﻲ', 'اسم'), ('ﻇﻞ', 'اسم'), ('ﺗﺴﺠﻴﻞ', 'اسم'), ('ﺣﻮﺍﻟﻲ', 'اسم'), ('ﺣﺎﻟﺔ', 'اسم'), ('ﺇﺟﻬﺎﺽ', 'اسم'), ('ﻳﻮﻣﻴﺎ', 'اسم'), ('ﻭﻗﺎﻝ', 'اسم'), ('ﺭﺋﻴﺲ', 'اسم'), ('ﺍﻟﺠﻤﻌﻴﺔ', 'اسم'), ('ﺍﻟﻤﻐﺮﺑﻴﺔ', 'اسم'), ('ﻟﻤﺤﺎﺭﺑﺔ', 'اسم'), ('ﺍﻹﺟﻬﺎﺽ', 'اسم'), ('ﺍﻟﺴﺮﻱ،', None),

##### For the machine learning approach : 

In [186]:
post_content4 = []
for i in range(len(all_posts)):

    arabic_text = all_posts["post_content3"][i]

    # Tokenize the sentence
    tokens = word_tokenize(arabic_text)

    pos_tags = nltk.pos_tag(tokens)
    # pos_tags = [nltk.pos_tag(e) for e in tokens]

    post_content4.append(pos_tags)

    if i == 1:
        # print("arabic_text : ", tokens)
        print("NLTK-based POS tagging: ", tagged_words)

NLTK-based POS tagging:  [('ﺃﻃﻠﻘﺖ', 'اسم'), ('ﻓﺪﺭﺍﻟﻴﺔ', 'اسم'), ('ﺭﺍﺑﻄﺔ', 'اسم'), ('ﺣﻘﻮﻕ', 'اسم'), ('ﺍﻟﻨﺴﺎﺀ', 'اسم'), ('ﺣﻤﻠﺔ', 'اسم'), ('ﺗﻮﺍﺻﻠﻴﺔ', 'اسم'), ('ﺗﻬﺪﻑ', 'اسم'), ('ﺑﺚ', 'اسم'), ('ﻓﻴﺪﻳﻮﻫﺎﺕ', 'اسم'), ('ﺗﺤﻤﻞ', 'اسم'), ('ﻗﺼﺼﺎ', 'اسم'), ('ﺣﻘﻴﻘﻴﺔ', 'اسم'), ('ﻟﻤﺸﺎﻛﻞ', 'اسم'), ('ﻧﺎﺗﺠﺔ', 'اسم'), ('ﺍﻟﻤﺪﻭﻧﺔ', 'اسم'), ('ﺍﻟﺤﺎﻟﻴﺔ،', None), ('ﻭﺗﻘﺮﺏ', 'اسم'), ('ﺍﻟﻤﺠﺘﻤﻊ', 'اسم'), ('ﺍﻵﺛﺎﺭ', 'اسم'), ('ﺗﺨﻠﻔﻬﺎ', 'اسم'), ('ﺍﻟﻤﻤﺎﺭﺳﺎﺕ', 'اسم'), ('ﻗﺒﻴﻞ', 'اسم'), ('ﺯﻭﺍﺝ', 'اسم'), ('ﺍﻟﻘﺎﺻﺮ،', None), ('ﻭﺯﻭﺍﺝ', 'اسم'), ('ﺍﻟﻔﺎﺗﺤﺔ،', None), ('ﻭﺍﻟﺘﻌﺪﺩ،', None), ('ﻭﺍﻟﻮﻻﻳﺔ،', None), ('ﻭﺍﻹﺭﺙ', 'اسم'), ('ﻭﺍﻗﺘﺴﺎﻡ', 'اسم'), ('ﺍﻟﻤﻤﺘﻠﻜﺎﺕ،', None), ('ﻭﻏﻴﺮﻫﺎ،', None), ('ﻭﻫﻲ', 'اسم'), ('ﺍﻟﺤﻤﻠﺔ', 'اسم'), ('ﺍﺧﺘﻴﺮﺕ', 'اسم'), ('ﺷﻌﺎﺭﺍﺕ', 'اسم'), ('ﻗﺒﻴﻞ', 'اسم'), ('ﺇﺻﻼﺡ', 'اسم'), ('ﺩﺍﺭﻱ', 'اسم'), ('ﺍﻟﻜﺒﻴﺮﺓ', 'اسم'), ('ﺇﺻﻼﺡ', 'اسم'), ('ﺩﺍﺭﻱ', 'اسم'), ('ﺍﻟﺼﻐﻴﺮﺓ،', None), ('ﻣﺪﻭﻧﺔ', 'اسم'), ('ﺃﺳﺮﺓ', 'اسم'), ('ﺿﺎﻣﻨﺔ', 'اسم'), ('ﻟﻠﻤﺴﺎﻭﺍﺓ', 'اسم'), ('ﻭﺍﻟﻌﺪﻝ', 'اسم'), ('ﻭﺍﻟﻤﺼﻠﺤﺔ', 'اسم'), ('ﺍﻟﻔﻀﻠﻰ', 'اسم'), ('ﻟﻸﻃﻔﺎﻝ', 'اسم'), ('ﻭﻓﻲ

##### **Synthesis**: In this question, I have Applied both rule-based and machine learning-based parts of speech tagging approaches for Arabic text.

-----------------
## 6. Apply NER Methods.

**Named Entity Recognition (NER)** : is a natural language processing task that aims to identify and classify named entities within a text into predefined categories such as persons, organizations, locations, dates, etc. There are various methods and tools available for performing NER on Arabic text. 

##### Using nltk :

In [188]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\driss\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\driss\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [190]:

for i in range(len(post_content4)):
    # Apply named entity recognition
    entities = nltk.chunk.ne_chunk(post_content4[i])

    if i == 0 : 
        print(post_content4[i])
        print(entities)


[('ﺿﻤﻦ', 'JJ'), ('ﺍﻟﺘﻘﺮﻳﺮ', 'NNP'), ('ﺗﺴﻠﻤﻪ', 'NNP'), ('ﻋﺰﻳﺰ', 'NNP'), ('ﺃﺧﻨﻮﺵ،', 'NNP'), ('ﺭﺋﻴﺲ', 'NNP'), ('ﺍﻟﺤﻜﻮﻣﺔ،', 'NNP'), ('ﻧﻬﺎﻳﺔ', 'NNP'), ('ﺷﻬﺮ', 'NNP'), ('ﺍﻟﻤﺎﺿﻲ،', 'NNP'), ('ﺍﻟﻬﻴﺌﺔ', 'NNP'), ('ﺍﻟﻤﻜﻠﻔﺔ', 'NNP'), ('ﺑﻤﺮﺍﺟﻌﺔ', 'NNP'), ('ﻣﺪﻭﻧﺔ', 'NNP'), ('ﺍﻷﺳﺮﺓ،', 'NNP'), ('ﺍﻟﻤﻘﺮﺭ', 'NNP'), ('ﺭﻓﻌﻪ', 'NNP'), ('ﺍﻟﻤﻠﻚ', 'NNP'), ('ﻣﺤﻤﺪ', 'NNP'), ('ﺍﻟﺴﺎﺩﺱ،', 'NNP'), ('ﻻﻓﺘﺎ', 'NNP'), ('ﻭﺟﻮﺩ', 'NNP'), ('ﻣﻘﺘﺮﺡ', 'NNP'), ('ﺗﻄﺮﻕ', 'NNP'), ('ﺗﻘﺴﻴﻢ', 'NNP'), ('ﺍﻟﺘﺮﻛﺔ', 'NNP'), ('ﺧﻼﻝ', 'NNP'), ('ﺗﺨﻮﻳﻞ', 'NNP'), ('ﺻﺎﺣﺐ', 'NNP'), ('ﺍﻟﻤﺎﻝ', 'NNP'), ('ﻭﺍﻷﺻﻮﻝ', 'NNP'), ('ﺳﻠﻄﺔ', 'NNP'), ('ﺍﺧﺘﻴﺎﺭ', 'NNP'), ('ﺍﻟﻨﻈﺎﻡ', 'NNP'), ('ﺍﻟﻤﻄﺒﻖ،', 'NNP'), ('ﺍﻟﻤﻴﺮﺍﺙ', 'NNP'), ('ﺍﻟﻮﺻﻴﺔ،', 'NNP'), ('ﺗﻮﺳﻴﻊ', 'NNP'), ('ﻧﻄﺎﻕ', 'NNP'), ('ﺍﻷﺧﻴﺮﺓ', 'NNP'), ('ﻟﺘﺸﻤﻞ', 'NNP'), ('ﺍﻷﺣﻔﺎﺩ', 'NNP'), ('ﻭﺭﻓﻊ', 'NNP'), ('ﺍﻟﻘﻴﻮﺩ', 'NNP'), ('ﺍﻟﻤﻔﺮﻭﺿﺔ', 'NNP'), ('ﻋﻠﻴﻬﺎ', 'NNP'), ('ﻭﻋﻠﻰ', 'NNP'), ('ﺍﻟﺮﻏﻢ', 'NNP'), ('ﺍﻟﻤﻘﺘﺮﺡ،', 'NNP'), ('ﻋﺪﻩ', 'NNP'), ('ﻛﺜﻴﺮﻭﻥ', 'NNP'), ('ﺗﺴﺮﻳﺒﺎ،', 'NNP'), ('ﻋﻤﻮﻣﺎ', 'NNP'), ('ﻭﺛﻴﻘﺔ', 'NNP'), ('ﺭﺍﺋﺠﺔ',

##### **Synthesis**: In this question, I have Explored named entity recognition methods to identify entities in Arabic text.

-------------------
### Conclusion
This project provided valuable insights into scraping and NLP pipeline techniques for Arabic text. By utilizing libraries like Scrapy and Beautiful Soup for scraping, and employing various NLP techniques including stemming, lemmatization, parts of speech tagging, and named entity recognition, we were able to process and analyze Arabic text from web sources effectively. The comparison between stemming and lemmatization highlighted the importance of considering linguistic context for accurate text processing. Additionally, the application of NER methods demonstrated the capability to extract valuable information from unstructured text data. Overall, this project serves as a foundational step towards building more advanced NLP applications for Arabic language processing.